In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load the packages we need

In [2]:
import numpy as np
import collections
import pandas as pd
import string
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [3]:
def buildModel(X_train,y_train,X_test,y_test,batch_size):
    print('Build model...')
    model = Sequential()
    #model.add(Embedding(max_features, 128, dropout=0.2))
    model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
    model.add(Dense(1))
    model.add(Activation('softmax'))

    model.compile(loss='categorical-crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10,
              validation_data=(X_test, y_test))
    score, acc = model.evaluate(X_test, y_test,
                                batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)

In [6]:
DEFAULT_FILE_PATH = './data/glove.6B.50d.txt'

In [7]:
def loadGloveToDict(filePath = DEFAULT_FILE_PATH):
    """
    Build the dictionary for the GLove.
    :return: a dicitonary with word as the key and the glove vector
    as the value(an array).
    """
    res = collections.defaultdict[list]
    with open(filePath) as f:
        for line in f:
            line = line.strip()
            if(not line):
                continue
            row = line.split()
            token = row[0]
            data = [float(x) for x in row[1:]]
            data = np.array(data)
            res[token] = data
    return res

In [8]:
def load_review_dataset(path):
    with open(path,'r',encoding='utf8') as f:
        content = f.readlines()
    res = []
    for line in content:
        a = line.strip('.').lower().split()
        b = []
        exclude = set(string.punctuation)
        for word in a:
            word = ''.join(ch for ch in word if ch not in exclude)
            b.append(word)
        res.append(b)
    # res = [line.strip('.').lower().split() for line in content]
    return res

In [9]:
def getAverageGloveFromDict(messages):
    """
    Build a dictionary and use that to get the glove vector
    :param messages: All the comments. Each row represent one comment
    :return: the average glove vectors for each comment
    """
    numRows = len(messages)
    resArrays = np.zeros([numRows,50])
    #Build the word dict from glove first
    gloveDict = loadGloveToDict()
    for i in range(numRows):
        message = messages[i]
        wordVector = np.zeros(50)
        for word in message:
            if(word in gloveDict):
                wordVector = np.stack([wordVector,gloveDict[word]])
        wordVector = np.mean(wordVector)
        resArrays[i,:] = wordVector
    return resArrays

In [10]:
def transform_text(messages, word_dictionary):
    numRows = len(messages)
    print(numRows)
    numCols = max(word_dictionary.values())+1
    print(numCols)
    resArray = np.zeros((numRows,numCols))
    for i in range(len(messages)):
        message = messages[i]
        for word in message:
            if(word in word_dictionary):
                col = word_dictionary[word]
                resArray[i][col] +=1
    return resArray

In [11]:
def loadTrainTestDataSet():
    train_review_path = './data/train.review.txt'
    train_rate_path = './data/train.rating.txt'
    dev_review_path = './data/dev.review.txt'
    dev_rate_path = './data/dev.rating.txt'
    test_review_path = './data/test.review.txt'
    test_rate_path = './data/test.rating.txt'
    review_path = './data/review_full.txt'
    rate_path = './data/rate_full.txt'
    messages = load_review_dataset(review_path)
    word_dict = getAverageGloveFromDict(messages)

    train_messages = load_review_dataset(train_review_path)
    train_resArray = transform_text(train_messages, word_dict)
    dev_messages = load_review_dataset(dev_review_path)
    dev_resArray = transform_text(dev_messages, word_dict)
    test_messages = load_review_dataset(test_review_path)
    test_resArray = transform_text(test_messages, word_dict)

    # resArray = transform_text(messages, word_dict)
    # print('res array shape ',resArray.shape)
    # glove = getAverageGlove(messages)
    # print('glove', glove.shape)
    # To construct the training dataset
    # numOfDataPoints = len(resArray)
    # numOfTraining = int(numOfDataPoints * 0.8)
    trainingDataX = train_resArray[:, ]
    testDataX = test_resArray[:, ]
    trainRateY = pd.read_csv(train_rate_path, header=None)
    testRateY = pd.read_csv(test_rate_path, header=None)
    # print(len(traRateY))
    trainingRateY = trainRateY[:][0]
    testRateY = testRateY[:][0]
    return (trainingDataX,trainingRateY,testDataX,testRateY)

In [12]:
def main():
    X_train,y_train,X_test,y_test = loadTrainTestDataSet()
    batch_size = 32
    buildModel(X_train, y_train, X_test, y_test, batch_size)
    

In [13]:
main()

TypeError: 'type' object is not subscriptable